In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import time
import torchvision
import sys
import os
import torch
import torch.nn.functional as F
import argparse
import cv2
import numpy as np
from glob import glob
import copy
from collections import OrderedDict
from tqdm import tqdm
import pandas as pd
from argparse import ArgumentParser
from torchvision.datasets import CIFAR10 as data_cifar
from torchvision.transforms import Normalize, ToTensor, Compose, RandomCrop, RandomHorizontalFlip, Resize
from easydict import EasyDict
from torch.optim.lr_scheduler import MultiStepLR

In [6]:
from google.colab import drive, files
drive.mount('/content/drive/', force_remount=True)
src = list(files.upload().values())[0]
open('resnet.py','wb').write(src)
import resnet 




Mounted at /content/drive/


Saving resnet.py to resnet.py


In [7]:
src = list(files.upload().values())[0]
open('dataloader_pytorch.py', 'wb').write(src)
import dataloader_pytorch

Saving dataloader_pytorch.py to dataloader_pytorch.py


In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/kaokore.zip", 'r')
zip_ref.extractall("./kaokore")

zip_ref.close()

In [14]:
class_label = ['noble', 'warrior', 'incarnation', 'commoner'] # class name
class_N = len(class_label) # class number
img_height, img_width = 128, 128 # image size
channel = 3
args = EasyDict({"lr": 0.1, "resume":False})

transform_train = Compose([
    Resize(128),
    RandomCrop(size = (128, 128), padding=4),
    RandomHorizontalFlip(),
    ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
print("1")
transform_test = Compose([
    ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
trainset = dataloader_pytorch.Kaokore(
    root='./kaokore/kaokore', split="train", transform=transform_train)
print("1")

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=32, shuffle=True, num_workers=2)

testset = dataloader_pytorch.Kaokore(
    root='./kaokore/kaokore', split="test", transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=32, shuffle=False, num_workers=2)
print("2")


1
1
2


In [0]:
print(trainset)

In [15]:
device = 'cuda'
net = resnet.ResNetKaokore().to(device)
net = torch.nn.DataParallel(net)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr ,
                      momentum=0.9, weight_decay = 6e-4)

# Training
scheduler = MultiStepLR(optimizer, milestones=[20, 30,40], gamma=0.1)

def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
      inputs, targets = inputs.to(device), targets.to(device)
      optimizer.zero_grad()
      outputs = net(inputs)
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()
      

      train_loss += loss.item()
      _, predicted = outputs.max(1)
      total += targets.size(0)
      correct += predicted.eq(targets).sum().item()
    print(correct/total)
def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    print(correct/total)
for epoch in range(0, 50):
    print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
    train(epoch)
    scheduler.step()
    test(epoch)


current lr 1.00000e-01

Epoch: 0
0.7597923548843795
0.7571157495256167
current lr 1.00000e-01

Epoch: 1
0.7673430863614913
0.6584440227703985
current lr 1.00000e-01

Epoch: 2
0.7810287871637565
0.7514231499051234
current lr 1.00000e-01

Epoch: 3
0.7855120339782916
0.7514231499051234
current lr 1.00000e-01

Epoch: 4
0.7899952807928268
0.7741935483870968
current lr 1.00000e-01

Epoch: 5
0.7954223690420009
0.7722960151802657
current lr 1.00000e-01

Epoch: 6
0.8140632373761209
0.7609108159392789
current lr 1.00000e-01

Epoch: 7
0.8069844266163284
0.7666034155597723
current lr 1.00000e-01

Epoch: 8
0.8279848985370458
0.7722960151802657
current lr 1.00000e-01

Epoch: 9
0.85181689476168
0.7703984819734345
current lr 1.00000e-01

Epoch: 10
0.8607833883907503
0.8406072106261859
current lr 1.00000e-01

Epoch: 11
0.8673902784332232
0.8368121442125237
current lr 1.00000e-01

Epoch: 12
0.8676262387918829
0.777988614800759
current lr 1.00000e-01

Epoch: 13
0.8732892873997169
0.8292220113851992
curre

In [16]:

test(0)

0.8747628083491461
